In [1]:
from pyspark.sql import functions as F
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1617198326784_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
play_count = spark.read.parquet('s3://17700project/play_counts.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
seed = 180000
(split_90_df, split_10_df) = play_count.randomSplit([0.9, 0.10], seed = seed)

# Let's cache these datasets for performance
training_df = split_90_df.cache()
test_df = split_10_df.cache()

print('Training: {0}, test: {1}\n'.format(
  training_df.count(), test_df.count())
)
training_df.show(3)
test_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training: 43537939, test: 4835646

+------------------+--------------------+-----+----------+----------+
|            songId|              userId|Plays|new_userId|new_songId|
+------------------+--------------------+-----+----------+----------+
|SOAAEKX12A6D4F7E4E|00732e30a5abe4bec...|    1|    950928|    359777|
|SOAAEKX12A6D4F7E4E|00a3ee4941f7799ac...|    5|    473004|    359777|
|SOAAEKX12A6D4F7E4E|0129ee80d24fc9fe4...|    2|    508986|    359777|
+------------------+--------------------+-----+----------+----------+
only showing top 3 rows

+------------------+--------------------+-----+----------+----------+
|            songId|              userId|Plays|new_userId|new_songId|
+------------------+--------------------+-----+----------+----------+
|SOAAEKX12A6D4F7E4E|03bfb186b1c7fbe92...|    1|    385109|    359777|
|SOAAEKX12A6D4F7E4E|059266e024833d7a2...|    1|     67922|    359777|
|SOAAEKX12A6D4F7E4E|065c1954997224976...|    1|    227403|    359777|
+------------------+----------

In [4]:
from pyspark.ml import Pipeline, Transformer, Estimator

class GlobalEstimator(Estimator):
    def __init__(self):
        super(GlobalEstimator, self).__init__()

    def _fit(self, df):
        avg_plays = df.agg({'Plays':'mean'}).collect()[0]['avg(Plays)']
        
        user_avg = training_df.groupBy('new_userId').agg({'Plays':'mean'})
        user_bias = user_avg.select(col('new_userId').alias('user_id'), (user_avg['avg(Plays)']-avg_plays).alias('user_bias'))
        user_bias = user_bias.rdd.map(lambda x: (x['user_id'], x['user_bias'])).collectAsMap()
        user_bias_broadcast = sc.broadcast(user_bias) 
        
        song_avg = training_df.groupBy('new_songId').agg({'Plays':'mean'})
        song_bias = song_avg.select(col('new_songId').alias('song_id'), (song_avg['avg(Plays)']-avg_plays).alias('song_bias'))
        song_bias = song_bias.rdd.map(lambda x: (x['song_id'], x['song_bias'])).collectAsMap()
        song_bias_broadcast = sc.broadcast(song_bias) 
        
        return GlobalTransformer(user_bias_broadcast, song_bias_broadcast, avg_plays)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
class GlobalTransformer(Transformer):
    """
    A custom Transformer which drops all columns that have at least one of the
    words from the banned_list in the name.
    """

    def __init__(self, user_bias_broadcast, song_bias_broadcast, avg_plays):
        super(GlobalTransformer, self).__init__()
        self.user_bias_broadcast = user_bias_broadcast
        self.song_bias_broadcast = song_bias_broadcast
        self.avg_plays = avg_plays
        
    def predict(self, user_id, song_id):
        try:
            return self.user_bias_broadcast.value[user_id - 1] + self.song_bias_broadcast.value[song_id - 1] + self.avg_plays
        except:
            try:
                return self.user_bias_broadcast.value[user_id - 1] + self.avg_plays
            except:
                try: 
                    return self.song_bias_broadcast.value[song_id - 1] + self.avg_plays
                except:
                    return self.avg_plays
            
    def _transform(self, df):
        predict_udf = udf(lambda user_id,song_id: self.predict(user_id, song_id), DoubleType())
        final_df = df.select(col('new_userId').alias('user_id'), col('new_songId').alias('song_id'), 'Plays',predict_udf(df.new_userId, df.new_songId).alias('pred_plays'))
        
        return final_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
global_model = GlobalEstimator()
GlobalPipeline = Pipeline()

GlobalPipeline.setStages([global_model])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pipeline_e712339b7077

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
reg_eval = RegressionEvaluator(predictionCol="pred_plays", labelCol="Plays", metricName="rmse")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.ml.tuning import CrossValidator
import time

strt_time = time.time()
crossval = CrossValidator(estimator = GlobalPipeline, evaluator = reg_eval, numFolds=10)
crossval.setEstimatorParamMaps([None])
cvModel = crossval.fit(training_df).bestModel
end_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
reg_eval.evaluate(cvModel.transform(test_df))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7.544260519760814

In [10]:
end_time - strt_time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

442.99070739746094

In [17]:
 + 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

unsupported operand type(s) for +: 'float' and 'dict'
Traceback (most recent call last):
TypeError: unsupported operand type(s) for +: 'float' and 'dict'



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
strt_time = time.time()
song_bias = sc.parallelize(cvModel.stages[0].song_bias_broadcast.value.items())
song_rating = song_bias.map(lambda x: (x[0], cvModel.stages[0].avg_plays + cvModel.stages[0].user_bias_broadcast.value[17]+x[1]))\
                        .sortBy(lambda x:x[1], ascending = False)
print(song_rating.take(5))
end_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(346214, 337.8544741353502), (380758, 222.85447413535022), (289693, 168.85447413535022), (108898, 157.4544741353502), (164886, 134.85447413535022)]

In [13]:
strt_time-end_time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-1.2321062088012695